# Home Assignment 5

## Problem 1 (3 points). Shamir secret sharing

Find the secret and initial polynomial in (n, 4)-threshold Shamir secret sharing scheme over $F_{19}$ for $(3,17); (17,5); (5,11); (11,4)$. Is it consistnet with $(4, 3)$?

In [26]:
import numpy as np

# in order to find the secret, we need to reconstruct the polynomial using the 3 points given
cubic_model = np.poly1d(np.polyfit([3, 17, 5, 11], [17, 5, 11, 4], 3))
print(cubic_model)

           3          2
-0.008433 x + 0.3894 x - 5.702 x + 30.83


In [27]:
# at x = 0 we should get our secret
cubic_model(0)

30.828869047619026

In [28]:
# mod our answer by 19, because we are in a ring
print(cubic_model(0) % 19)

11.828869047619026


## Problem 2 (2* points). Schnorr ring signature

For the elliptic curve $y^2=x^3+7$ over $F_{127}$ and the hash function $H() = SHA256()\%127$. Consider a subgroup of point on a curve with a generator $G = (19, 32)$.

Suppose Alice has a pricte key $a = 17$ and Bob has a prite key $b = 29$.

1. (0 points) Compute public keys.
2. (2* points) Generate Schnorr's ring signature for $m=77$ on Alice's behalf with $k=101$.

## Problem 3 (3 points). Bitcoin data

Consider tx=21e8fd77b81b6c4cf6723c10ea83acae77728f188ee6f4cd97ae114665b5fc0f (block height: 478994, tx index/position: 1603). Trace the transaction back to address 12t9YDPgwueZ9NyMgw519p7AA8isjr6SMw.

Use blockexplorer Python library, Bitcoin RPC or their equivalent alternative.

In [1]:
from blockchain import blockexplorer

def get_block(block_id):
    return blockexplorer.get_block(block_id)

# loading block
block = get_block(block_id="478994")

In [2]:
block.transactions[1603]

In [3]:
def get_compressed_form_of_public_key(uncompressed_pubkey):
    x = uncompressed_pubkey[0:(int(len(uncompressed_pubkey) / 2))]
    y = uncompressed_pubkey[(int(len(uncompressed_pubkey) / 2)):]
    if int(y, 16) % 2 == 0:
        pubkey = '2' + x
    else:
        pubkey = '3' + x
    return pubkey

# get public keys compressed
def get_pubkeysc(pubkeys):
    pubkeysc = []
    for pubkey in pubkeys:
        pubkeyc = get_compressed_form_of_public_key(pubkey)
        pubkeysc.append(pubkeyc)
    return pubkeysc

def get_hashes_hex(deserialized_script):
    hashes = deserialized_script['hashes']
    hashes_hex = []
    for hash in hashes:
        hashes_hex.append(hash.hex())
    return hashes_hex

def get_pubkeys_hex(deserialized_script):
    pubkeys = deserialized_script['keys']
    pubkeys_hex = []
    for pubkey in pubkeys:
        pubkeys_hex.append(pubkey.hex())
    return pubkeys_hex

from bitcoinaddress import Address
from bitcoinaddress.util import hash160, ecdsa_secp256k1, checksum
from bitcoinaddress import segwit_addr

# SEGWIT_PREFIX = b'\x05'
# P2SH_P2WPKH
def generate_public_address_3(pubkeyc, prefix= b'\x05'):
    hex_str = bytearray.fromhex(pubkeyc)
    prefix_redeem = b'\x00\x14'
    redeem_script = hash160(prefix_redeem + hash160(hex_str).digest()).digest()  # 20 bytes
    m = prefix + redeem_script
    c = checksum(m)
    return base58.b58encode(m + c).decode('utf-8')

def get_public_addresses_3(pubkeysc):
    pub_addresses_3 = []
    for pubkeyc in pubkeysc:
        adr = generate_public_address_3(pubkeyc)
        pub_addresses_3.append(adr)
    return pub_addresses_3

def generate_public_address_1_compressed(pubkeyc):
    hex_str = bytearray.fromhex(pubkeyc)
    key_hash = '00' + hash160(hex_str).hexdigest()
    sha = hashlib.sha256()    
    sha.update(bytearray.fromhex(key_hash))
    checksum = sha.digest()
    sha = hashlib.sha256()
    sha.update(checksum)
    checksum = sha.hexdigest()[0:8]
    return base58.b58encode( bytes(bytearray.fromhex(key_hash + checksum))).decode('utf-8')

def get_public_addresses_1_compressed(pubkeysc):
    pub_addresses_1_c = []
    for pubkeyc in pubkeysc:
        adr = generate_public_address_1_compressed(pubkeyc)
        pub_addresses_1_c.append(adr)
    return pub_addresses_1_c

def generate_public_address_bech32_P2WPKH(pubkeyc, bech32_prefix=Address.MainNet.BECH32_PREFIX):
    hex_str = bytearray.fromhex(pubkeyc)
    redeem_script_P2WSH = hash160(hex_str).digest()
    adr =  str(segwit_addr.encode(bech32_prefix, Address.WITNESS_VERSION, redeem_script_P2WSH))
    return adr

def get_public_addresses_bech32_P2WPKH(pubkeysc):
    addresses = []
    for pubkeyc in pubkeysc:
        adr = generate_public_address_bech32_P2WPKH(pubkeyc)
        addresses.append(adr)
    return addresses

def generate_public_address_bech32_P2WSH(pubkeyc, bech32_prefix=Address.MainNet.BECH32_PREFIX):
    hex_str = bytearray.fromhex(pubkeyc)
    redeem_script_P2WSH = hash160(hex_str).digest()
    adr =  str(segwit_addr.encode(bech32_prefix, Address.WITNESS_VERSION, redeem_script_P2WSH))
    return adr

def get_public_addresses_bech32_P2WSH(pubkeysc):
    addresses = []
    for pubkeyc in pubkeysc:
        adr = generate_public_address_bech32_P2WSH(pubkeyc)
        addresses.append(adr)
    return addresses

from bitcoinlib.transactions import script_deserialize

def generate_outputs_dict(outputs):
    shortened_outputs = {}
    for output in outputs:
        deserialized_script = script_deserialize(output.script)
        # hash160_pubkeys = get_hashes_hex(deserialized_script)

        pubkeys_hex = get_pubkeys_hex(deserialized_script)
        pubkeysc_hex = get_pubkeysc(pubkeys_hex)
        public_addresses_3 = get_public_addresses_3(pubkeysc_hex)
        public_addresses_1_compressed = get_public_addresses_1_compressed(pubkeysc_hex)
        public_addresses_bc1_p2wpkh = get_public_addresses_bech32_P2WPKH(pubkeysc_hex)
        public_addresses_bc1_p2wsh = get_public_addresses_bech32_P2WSH(pubkeysc_hex)

        shortened_outputs.update({
            output.n: {
                "pubkeys": pubkeys_hex,
                "pubkeysc": pubkeysc_hex,
                "address": output.address,
                "address 1 compressed": public_addresses_1_compressed,
                "address 3": public_addresses_3,
                "address bc1 P2WPKH ": public_addresses_bc1_p2wpkh,
                "address bc1 P2WSH ": public_addresses_bc1_p2wsh,
                "locking_script_type": deserialized_script['script_type'],
                "script": output.script,
                "value": output.value,
                "m": deserialized_script['number_of_sigs_m'],
                "n": deserialized_script['number_of_sigs_n']
            }
        })

    return shortened_outputs

def generate_inputs_dict(inputs):
    shortened_inputs = {}
    for id_input in range(len(inputs)):
        input = inputs[id_input]
        try:
            if input.value == 1215261 or input.value == 315637178:
                print(dir(input))
                print(input.script)
                print(input.script_sig)
                print(input.type)
                print(input.address)
                print(input.value)
                print(input.tx_index)
                print(input.sequence)
                print("-------------------------")
        except:
            pass
        try:
            deserialized_script_sig = script_deserialize(input.script_sig)
            deserialized_script = script_deserialize(input.script)

            shortened_inputs.update({
                id_input: {
                    "pubkeys": get_pubkeys_hex(deserialized_script_sig),
                    "address when it was output": input.address,
                    "script": input.script,
                    "script_type": deserialized_script['script_type'] + '_' + deserialized_script_sig['script_type'],
                    "value": input.value,
                    "script_sig": input.script_sig,
                    "m": deserialized_script_sig['number_of_sigs_m'],
                    "n": deserialized_script_sig['number_of_sigs_n']
                }
            })

        except:
            # coinbase transaction
            pass
    return shortened_inputs

def load_txs_from_block(block, min_tx_index=0, max_tx_index=100):
    txs = {}    
    for id_tx in range(min_tx_index, min(max_tx_index + 1, len(block.transactions))):
        transaction = block.transactions[id_tx]
        txs.update({id_tx: {"hash": transaction.hash,
                            "inputs": generate_inputs_dict(transaction.inputs),
                            "outputs": generate_outputs_dict(transaction.outputs)}
                    })

    return txs

In [4]:
txs = load_txs_from_block(block, min_tx_index = 1600, max_tx_index = 1604)

In [7]:
txs[1603]

{'hash': '21e8fd77b81b6c4cf6723c10ea83acae77728f188ee6f4cd97ae114665b5fc0f',
 'inputs': {0: {'pubkeys': ['028e759c84083746cbe9eeafd586886d07ff1bd195404e021a438f128d575b1fbe'],
   'address when it was output': '1A6ezvhzGmCqNmGTTzxphLkByuJfjbuwxr',
   'script': '76a91463caee7d58a7fc7732f491dbfd060cd566e5a5cf88ac',
   'script_type': 'p2pkh_sig_pubkey',
   'value': 687748200,
   'script_sig': '483045022100d6e99596369497a9c6409a9423a4492f087a34a749454c7d0ca55b9353b8a31602206ce17df7a9d3ab1daf9fc355ad57120eddd9c19d1e25a6c1099e35481d6abaee0121028e759c84083746cbe9eeafd586886d07ff1bd195404e021a438f128d575b1fbe',
   'm': 1,
   'n': 1}},
 'outputs': {0: {'pubkeys': [],
   'pubkeysc': [],
   'address': '1A6ezvhzGmCqNmGTTzxphLkByuJfjbuwxr',
   'address 1 compressed': [],
   'address 3': [],
   'address bc1 P2WPKH ': [],
   'address bc1 P2WSH ': [],
   'locking_script_type': 'p2pkh',
   'script': '76a91463caee7d58a7fc7732f491dbfd060cd566e5a5cf88ac',
   'value': 174637100,
   'm': 1,
   'n': 1},
  1: 

In [17]:
address = blockexplorer.get_address("1A6ezvhzGmCqNmGTTzxphLkByuJfjbuwxr")

input_dicts = []
for transaction in address.transactions:
    input_dicts.append(generate_inputs_dict(transaction.inputs))

In [18]:
input_dicts

[{0: {'pubkeys': ['028e759c84083746cbe9eeafd586886d07ff1bd195404e021a438f128d575b1fbe'],
   'address when it was output': '1A6ezvhzGmCqNmGTTzxphLkByuJfjbuwxr',
   'script': '76a91463caee7d58a7fc7732f491dbfd060cd566e5a5cf88ac',
   'script_type': 'p2pkh_sig_pubkey',
   'value': 174637100,
   'script_sig': '47304402200a9f953e824b6bd5fb1c6d9b804ca80b1020661ff4aa4d4602644093a496a77602203f7efb5a21a558736eae68f772b0d7eb55f7d77fec21a7add08925a2a4db1dd30121028e759c84083746cbe9eeafd586886d07ff1bd195404e021a438f128d575b1fbe',
   'm': 1,
   'n': 1},
  1: {'pubkeys': ['0385c0eb8f947a961922f75c27fa56385f891a6bcb7c5528f433a31fad55031a4d'],
   'address when it was output': '164Cg2p4QQ16VJGoBgiST5v2yDZtrvPgan',
   'script': '76a91437739ef2ea2147a23639b7f688aa64c91283deac88ac',
   'script_type': 'p2pkh_sig_pubkey',
   'value': 625191500,
   'script_sig': '473044022069d46950d612066115d82831f0a3f416b700aca7947d43d0c5892c40a0d8f5a3022063626e6f2574ea18304765694e1225865595269a56c183387b2c0f643d2a3d3601210385

In [19]:
address = blockexplorer.get_address("1FQQ86tMuvhQ4Ruyggbb8j7iaNfUZ69gpY")
input_dicts = []
for transaction in address.transactions:
    input_dicts.append(generate_inputs_dict(transaction.inputs))

In [20]:
input_dicts

[{0: {'pubkeys': ['021efed9625a32eef4cc68d1a1af6eb57708f769df307d039dc6e76253974bc80d'],
   'address when it was output': '1FQQ86tMuvhQ4Ruyggbb8j7iaNfUZ69gpY',
   'script': '76a9149dfea13643f2d099c6148d002b4a002cb853395688ac',
   'script_type': 'p2pkh_sig_pubkey',
   'value': 871529348,
   'script_sig': '483045022100fe6b64e302503d906cf16cab9b2949a17fd4b527d5c61be9ff7d8899886c303602200bc98d93bf15db47bcd68459a416306355af338f9dedfa8df56687bda0f6df140121021efed9625a32eef4cc68d1a1af6eb57708f769df307d039dc6e76253974bc80d',
   'm': 1,
   'n': 1}},
 {0: {'pubkeys': ['026eace100993a8d73f3ce1761fcfd359c015b04a9b9ebbcf9a33e9a9ca81e9c2f'],
   'address when it was output': '12t9YDPgwueZ9NyMgw519p7AA8isjr6SMw',
   'script': '76a91414a477964ed719135d1598da348a858b18b44fd588ac',
   'script_type': 'p2pkh_sig_pubkey',
   'value': 17124673,
   'script_sig': '483045022100f466e70d53728e4cd222367f9236fbed484590bec57bae4a46898379bc6c34b8022053d8a12c030c7ae941ec46af7b02b32e6ea1493bf5fe83c017c38795a8a9e79b0121

Found original: 12t9YDPgwueZ9NyMgw519p7AA8isjr6SMw

## Problem 4 (3* points). Shared send untangling

1. (1* point) Simplify transaction (see Section 1 \[1\])
* 409803bb5e124fd028c0482027c7722e84ce55b78204b279d3a44aba5e7c1698
* 9ea6fdad2ea5870f5b17cb28587263f90cd7ec5c67f35d395a318186e5c79333
* c38aac9910f327700e0f199972eed8ea7c6b1920e965f9cb48a92973e7325046
* 659135664894e50040830edb516a76f704fd2be409ecd8d1ea9916c002ab28a2
2. (2* points) What are these transactions types (see Theorem 1)?

\[1\] https://bitfury.com/content/downloads/bitfury_whitepaper_shared_send_untangling_in_bitcoin_8_24_2016.pdf

## Problem 5 (4 points). ERC20 data

Consider (scam?) token 0xcae72a7a0fd9046cf6b165ca54c9e3a3872109e0 from Ethereum mainnet. How many tokens where transfered in Dec-2020?

In [1]:
import requests, json
from web3 import Web3, HTTPProvider

In [2]:
web3_url = "https://mainnet.infura.io/v3/f25af56bef3b4460a475fd5ddc05b227"
w3 = Web3(Web3.HTTPProvider(web3_url))

In [3]:
address = Web3.toChecksumAddress('0xcae72a7a0fd9046cf6b165ca54c9e3a3872109e0') 
# ABI (Application Binary Interface)
abi = '[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"owner","type":"address"},{"indexed":true,"internalType":"address","name":"spender","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Approval","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"previousOwner","type":"address"},{"indexed":true,"internalType":"address","name":"newOwner","type":"address"}],"name":"OwnershipTransferred","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"from","type":"address"},{"indexed":true,"internalType":"address","name":"to","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Transfer","type":"event"},{"inputs":[{"internalType":"address","name":"owner","type":"address"},{"internalType":"address","name":"spender","type":"address"}],"name":"allowance","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"approve","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"account","type":"address"}],"name":"balanceOf","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"burn","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"decimals","outputs":[{"internalType":"uint8","name":"","type":"uint8"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"subtractedValue","type":"uint256"}],"name":"decreaseAllowance","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"addedValue","type":"uint256"}],"name":"increaseAllowance","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"isOwner","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"account","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"mint","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"name","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"owner","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"renounceOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"symbol","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"totalSupply","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"recipient","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"transfer","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"sender","type":"address"},{"internalType":"address","name":"recipient","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"transferFrom","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"newOwner","type":"address"}],"name":"transferOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"}]'
contract_instance = w3.eth.contract(address=address, abi=abi)

# read state:
print(contract_instance.functions.name().call())

AnRKey X


In [4]:
# import dependencies
import pickle
from tqdm import tqdm

# request the latest block number
ending_blocknumber = w3.eth.blockNumber

# latest block number minus 100 blocks
starting_blocknumber = w3.eth.blockNumber - 100

# filter through blocks and look for transactions involving this address
blockchain_address = address

# create an empty dictionary we will add transaction data to
tx_dictionary = {}

def getTransactions(start, end, address):
    '''This function takes three inputs, a starting block number, ending block number
    and an Ethereum address. The function loops over the transactions in each block and
    checks if the address in the to field matches the one we set in the blockchain_address.
    Additionally, it will write the found transactions to a pickle file for quickly serializing and de-serializing
    a Python object.'''
    print(f"Started filtering through block number {start} to {end} for transactions involving the address - {address}...")
    for x in tqdm(range(start, end)):
        block = w3.eth.getBlock(x, True)
        for transaction in block.transactions:
            if transaction['to'] == address or transaction['from'] == address:
                with open("transactions.pkl", "wb") as f:
                    hashStr = transaction['hash'].hex()
                    tx_dictionary[hashStr] = transaction
                    pickle.dump(tx_dictionary, f)
                f.close()
    print(f"Finished searching blocks {start} through {end} and found {len(tx_dictionary)} transactions")
    return tx_dictionary
    

I will not use the aforementioned method, because it it too slow. I downloaded all transactions list from Etherscan from the ERC20 contract and filter the 'Transfer' transactions.

In [50]:
import pandas

df = pandas.read_csv("transactions.csv", index_col=False) # downloaded from etherscan, december transactions
df

,Txhash,Blockno,UnixTimestamp,DateTime,From,To,ContractAddress,Value_IN(ETH),Value_OUT(ETH),CurrentValue @ $1539.95/Eth,TxnFee(ETH),TxnFee(USD),Historical $Price/Eth,Status,ErrCode,Method
0,0x7ccc0f887845141a13bad9d215e8168076505d5b1f94...,11457978,1608039708,2020-12-15 13:41:48,0xb4866eed83d9c257c814d78c2170f1a428b5110b,0xcae72a7a0fd9046cf6b165ca54c9e3a3872109e0,NaN,0.0,0,0.0,0.002448,3.769282,589.17,NaN,NaN,Approve
1,0x150b9979884c340bfee39c148452671323513df6a526...,11460728,1608076586,2020-12-15 23:56:26,0x0fe72b33a881e9b4e40bbe1c49ba24cc800a1f4b,0xcae72a7a0fd9046cf6b165ca54c9e3a3872109e0,NaN,0.0,0,0.0,0.002961,4.559479,589.17,NaN,NaN,Approve
2,0x7287774ade25fe8e6041ede1410d8237aeefcc81f515...,11460796,1608077441,2020-12-16 00:10:41,0x0fe72b33a881e9b4e40bbe1c49ba24cc800a1f4b,0xcae72a7a0fd9046cf6b165ca54c9e3a3872109e0,NaN,0.0,0,0.0,0.002043,3.146688,637.74,NaN,NaN,Approve
3,0x2a714d6de8d3e04c6103b461303b63b6845508b42ba6...,11460815,1608077707,2020-12-16 00:15:07,0x0fe72b33a881e9b4e40bbe1c49ba24cc800a1f4b,0xcae72a7a0fd9046cf6b165ca54c9e3a3872109e0,NaN,0.0,0,0.0,0.002107,3.245085,637.74,NaN,NaN,Approve
4,0x4cefffbfea31bda18eb20c77ad622e18333c50ed3084...,11461644,1608088590,2020-12-16 03:16:30,0x4775259e2cb64ad5bda0a396568732fc655c593c,0xcae72a7a0fd9046cf6b165ca54c9e3a3872109e0,NaN,0.0,0,0.0,0.003797,5.847326,637.74,NaN,NaN,Transfer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2273,0x2e39f044bd41649dc754ca13145eeda811c4d01c1d2f...,11562714,1609427942,2020-12-31 15:19:02,0xbbe3aba57ac2a5dc36ebb9769fcbd53fd5153f93,0xcae72a7a0fd9046cf6b165ca54c9e3a3872109e0,NaN,0.0,0,0.0,0.004895,7.538563,737.15,NaN,NaN,Approve
2274,0x26624bea4d028d9679cac5605afbac935c44dda9c5c7...,11563508,1609438782,2020-12-31 18:19:42,0x06af94eb30d292f3b910b6b93222ac16fa000819,0xcae72a7a0fd9046cf6b165ca54c9e3a3872109e0,NaN,0.0,0,0.0,0.002692,4.146210,737.15,NaN,NaN,Approve
2275,0xa8e7056a26899c372387f356788fe33eabbe88daabd2...,11563649,1609440685,2020-12-31 18:51:25,0x6f3730a184054b96786dda5387ea70a9967732d1,0xcae72a7a0fd9046cf6b165ca54c9e3a3872109e0,NaN,0.0,0,0.0,0.001217,1.873867,737.15,NaN,NaN,Transfer
2276,0x7198040a3f4c0bd50fe97dbd1c8eb18751fb470a139d...,11563722,1609441689,2020-12-31 19:08:09,0xb44d3b0507e69f7fd3683fb4ec3bdbb7cca0ae25,0xcae72a7a0fd9046cf6b165ca54c9e3a3872109e0,NaN,0.0,0,0.0,0.002937,4.523138,737.15,NaN,NaN,Approve


In [51]:
df = df[df['Method'] == 'Transfer']

In [53]:
df

,Txhash,Blockno,UnixTimestamp,DateTime,From,To,ContractAddress,Value_IN(ETH),Value_OUT(ETH),CurrentValue @ $1539.95/Eth,TxnFee(ETH),TxnFee(USD),Historical $Price/Eth,Status,ErrCode,Method
4,0x4cefffbfea31bda18eb20c77ad622e18333c50ed3084...,11461644,1608088590,2020-12-16 03:16:30,0x4775259e2cb64ad5bda0a396568732fc655c593c,0xcae72a7a0fd9046cf6b165ca54c9e3a3872109e0,NaN,0.0,0,0.0,0.003797,5.847326,637.74,NaN,NaN,Transfer
5,0x3bd89b7fd0acf9df3743b8bfe12c75f51173265d1e90...,11461677,1608089180,2020-12-16 03:26:20,0xaf9981d21d212d9cb5f2c9df6b56029737ac53e7,0xcae72a7a0fd9046cf6b165ca54c9e3a3872109e0,NaN,0.0,0,0.0,0.003873,5.964457,637.74,NaN,NaN,Transfer
6,0x5b1909ab79b6513aefcef1bd530e74d30be5bf6ddfd2...,11461710,1608089519,2020-12-16 03:31:59,0xaf9981d21d212d9cb5f2c9df6b56029737ac53e7,0xcae72a7a0fd9046cf6b165ca54c9e3a3872109e0,NaN,0.0,0,0.0,0.001699,2.615867,637.74,NaN,NaN,Transfer
7,0xffe4322c3bdc48efa2b3332df04e31cb39d17200d957...,11461718,1608089696,2020-12-16 03:34:56,0x1bfeaf74553325fc7064f6d19287cb9374ce3f05,0xcae72a7a0fd9046cf6b165ca54c9e3a3872109e0,NaN,0.0,0,0.0,0.003450,5.312273,637.74,NaN,NaN,Transfer
8,0x7735c24a0a7584cb94a679de4cb8ef864c777a955867...,11461758,1608090107,2020-12-16 03:41:47,0x3e9afaa4a062a49d64b8ab057b3cb51892e17ecb,0xcae72a7a0fd9046cf6b165ca54c9e3a3872109e0,NaN,0.0,0,0.0,0.004413,6.795541,637.74,NaN,NaN,Transfer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2261,0x7a18e80993465fa9898c7cded940ee468aa63a9f6ec9...,11556561,1609346669,2020-12-30 16:44:29,0x6f3730a184054b96786dda5387ea70a9967732d1,0xcae72a7a0fd9046cf6b165ca54c9e3a3872109e0,NaN,0.0,0,0.0,0.002519,3.879233,752.53,NaN,NaN,Transfer
2266,0x1cf778fb2778b3ab9bc8a160e4518a93c73da9256a4e...,11558996,1609378817,2020-12-31 01:40:17,0x4e0e8e636d01216855946aaa163d7fb73c86d070,0xcae72a7a0fd9046cf6b165ca54c9e3a3872109e0,NaN,0.0,0,0.0,0.002154,3.316627,737.15,NaN,NaN,Transfer
2267,0xab9daab3a2e67b006abf480c5649a58f4dc06295e478...,11559000,1609378868,2020-12-31 01:41:08,0x3e9afaa4a062a49d64b8ab057b3cb51892e17ecb,0xcae72a7a0fd9046cf6b165ca54c9e3a3872109e0,NaN,0.0,0,0.0,0.005184,7.982676,737.15,NaN,NaN,Transfer
2272,0xfba9c1dd6042654388f65e388af24bb2ab05af00b663...,11561630,1609413763,2020-12-31 11:22:43,0x141fef8cd8397a390afe94846c8bd6f4ab981c48,0xcae72a7a0fd9046cf6b165ca54c9e3a3872109e0,NaN,0.0,0,0.0,0.003894,5.997277,737.15,NaN,NaN,Transfer


In [54]:
transactions = df['Txhash'].tolist()

In [55]:
transactions

['...']

In [56]:
len(transactions)

613

In [57]:
import json
from web3_input_decoder import decode_function

tokens_transfer_in_total = 0.0

for i in range(len(transactions)):
    print(i, " transaction decoding...")
    tx = w3.eth.getTransaction(transactions[i])
    print("Transaction: ", transactions[i])
    try:
        decoded_data = decode_function(json.loads(abi), tx['input'])
        amount = decoded_data[1][2]
        tokens_transfer_in_total += (amount / (10**18))
    except:
        print("Warning! Error encountered during contract execution")
    print("Sum right now: ", tokens_transfer_in_total)

0  transaction decoding...
Transaction:  0x4cefffbfea31bda18eb20c77ad622e18333c50ed3084599e045b69f773a0725b
Sum right now:  4800.0
1  transaction decoding...
Transaction:  0x3bd89b7fd0acf9df3743b8bfe12c75f51173265d1e90c25b1a37b9250d6788b2
Sum right now:  4801.0
2  transaction decoding...
Transaction:  0x5b1909ab79b6513aefcef1bd530e74d30be5bf6ddfd2dd5083af571f645ebf12
Sum right now:  9600.0
3  transaction decoding...
Transaction:  0xffe4322c3bdc48efa2b3332df04e31cb39d17200d957fb5eab0dae491929de11
Sum right now:  14400.0
4  transaction decoding...
Transaction:  0x7735c24a0a7584cb94a679de4cb8ef864c777a95586793e1493f6b41a5f3af56
Sum right now:  14640.0
5  transaction decoding...
Transaction:  0x2391ff6b43266e01f161b7812631c41c4d1086449781fd4aa011d41899d537ff
Sum right now:  18719.99999999
6  transaction decoding...
Transaction:  0x88b10e2cea0b2852b90666d4e44ffa73100c43285f90c4b623a08f575728a11a
Sum right now:  18720.99999999
7  transaction decoding...
Transaction:  0x0073e5f0dac622ef7681a3

In [58]:
print(tokens_transfer_in_total)

42321877.88996789


## Problem 6 (3* points). BigQuery

Consider (scam?) token 0xcae72a7a0fd9046cf6b165ca54c9e3a3872109e0 from Ethereum mainnet. Compute the total amount of tokens transfered. Use bigquery-public-data.crypto_ethereum (https://ethereum-etl.readthedocs.io/en/latest/google-bigquery/).